<a href="https://colab.research.google.com/github/Nagendra0228/INS-Task-2/blob/main/INS_TASK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🔹 Step 1: Install Required Libraries**

In [ ]:
!pip install --upgrade cryptography


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.2 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.2 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.2 which is incompatible.


# **🔹 Step 2: Generate and Store Keys (RSA & AES)**

In [ ]:
import os
from cryptography.hazmat.primitives.asymmetric import rsa, dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Function to generate RSA Key Pair
def generate_rsa_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()

    # Save Private Key
    with open("/content/private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))

    # Save Public Key
    with open("/content/public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

    return private_key, public_key

# Function to generate AES Key
def generate_aes_key():
    aes_key = os.urandom(32)  # 256-bit AES key
    with open("/content/aes_key.bin", "wb") as f:
        f.write(aes_key)
    return aes_key

# Generate RSA and AES Keys
private_key, public_key = generate_rsa_keys()
aes_key = generate_aes_key()
print("✅ RSA & AES keys generated and stored in /content/")


✅ RSA & AES keys generated and stored in /content/


# **🔹 Step 3: Secure Key Exchange using Diffie-Hellman**

In [ ]:
# Generate Diffie-Hellman Parameters and Private Keys
parameters = dh.generate_parameters(generator=2, key_size=2048)
private_key_A = parameters.generate_private_key()
private_key_B = parameters.generate_private_key()

# Generate Public Keys
public_key_A = private_key_A.public_key()
public_key_B = private_key_B.public_key()

# Compute Shared Secret
shared_secret_A = private_key_A.exchange(public_key_B)
shared_secret_B = private_key_B.exchange(public_key_A)

assert shared_secret_A == shared_secret_B  # Verify they are the same
print("✅ Secure Diffie-Hellman Key Exchange Successful!")


✅ Secure Diffie-Hellman Key Exchange Successful!


# **🔹 Step 4: Implement Key Revocation System**

In [ ]:
revoked_keys = set()

# Function to revoke a key
def revoke_key(key_id):
    revoked_keys.add(key_id)
    print(f"🚨 Key {key_id} has been revoked.")

# Function to check if a key is revoked
def is_key_revoked(key_id):
    return key_id in revoked_keys

# Example: Revoking a key
revoke_key("key-123")
print("Is key revoked? ", is_key_revoked("key-123"))  # Output: True


🚨 Key key-123 has been revoked.
Is key revoked?  True


# **🔹 Step 5: Encryption & Decryption (AES)**

In [ ]:
# Encrypt Data with AES
def encrypt_data_aes(data, aes_key):
    iv = os.urandom(16)  # 128-bit IV
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padded_data = data.ljust((len(data) // 16 + 1) * 16).encode()  # Padding
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return iv + encrypted_data

# Decrypt Data with AES
def decrypt_data_aes(encrypted_data, aes_key):
    iv, encrypted_content = encrypted_data[:16], encrypted_data[16:]
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(encrypted_content) + decryptor.finalize()
    return decrypted_data.strip().decode()

# Test Encryption & Decryption
message = "Confidential Data"
encrypted_msg = encrypt_data_aes(message, aes_key)
print("🔒 Encrypted:", encrypted_msg)

decrypted_msg = decrypt_data_aes(encrypted_msg, aes_key)
print("🔓 Decrypted:", decrypted_msg)


🔒 Encrypted: b'\xa9\x1e\xae0H%IaH\x84\xb9\xf3/ff\xe5\xfe\x94\xff\xa7H<D\xd2:\xed\x0biY\xcb\xb6\xe3\xae\x9ee[\x1dS\xa2"\xf7\x08\xba\xf6\xd9\xf2\xa11'
🔓 Decrypted: Confidential Data
